In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
def load_hands(hand_path):
    hand_list = list()
    for file in os.listdir(hand_path):
        if file.endswith('.log'):
            with open(hand_path+file) as f:
                hand_list.extend( f.readlines() )
    return hand_list

def split_hand(any_hand):
    sh = any_hand.split(':')
    return sh

def hole_card_simplifier(hole_cards):
    clean_hands = list()
    true_hands = list()
    for hand in hole_cards:
        ord_hand = ''.join(sorted([hand[0], hand[2]], key=card_ranks.__getitem__))
        if hand[1] == hand[3]:
            ord_hand += 's'
        elif hand[1] != hand[3] and hand[0] != hand[2]:
            ord_hand += 'o'
        clean_hands.append(ord_hand)
    return clean_hands

def flop_texture(flop=''):
    ord_flop = ''
    try:
        ord_flop = ''.join(sorted([flop[0], flop[2], flop[4]], key=card_ranks.__getitem__))
        if flop[1] == flop[3] == flop[5]:
            ord_flop += 's'
        elif flop[1] == flop[3] or flop[1] == flop[5] or flop[3] == flop[5]:
            ord_flop += 't'
        else:
            ord_flop += 'r'
        return ord_flop
    except:
        return ''
    
def get_actions(line_of_actions): #second element of split hand list
    raise_size = str()
    actions = list()
    for char in line_of_actions:
        if char == 'r':
            if len(raise_size) > 0:
                actions.append(raise_size.replace('r', ''))
                raise_size = 'r'
            else:
                raise_size += char
        elif char.isdigit():
            raise_size += char
        elif char == 'f':
            if len(raise_size) > 0:
                actions.append(raise_size.replace('r', ''))
                actions.append('f')
                raise_size = str()
            else:
                actions.append('f')
        elif char == 'c':
            if len(raise_size) > 0:
                actions.append(raise_size.replace('r', ''))
                actions.append('c')
                raise_size = str()
            else:
                actions.append('c')
        elif char == '/':
            if len(raise_size) > 0:
                actions.append(raise_size)
                actions.append('next_street')
                raise_size = str()
            else:
                actions.append('next_street')
    
    return actions

def get_pluribus_actions(bet_order, action_sequence):
    raise_num = 0
    street = 0
    plur_bets = 0
    
    plur_r_pf = ''; plur_3b_pf = ''; plur_bets_flop = ''
    plur_f_when_3b = ''; 
    plur_c_open = ''; plur_c_when_3b = ''; plur_cold_c_3b = ''
    plur_r_size = ''; plur_3b_size = ''
    plur_able_to_3b = 'Pluribus unable to 3bet'
    
    for act in action_sequence:
        if bet_order[0] == 'Pluribus' and raise_num == 1 and street == 0 and plur_bets == 0:
            plur_able_to_3b = 'Pluribus able to 3bet'
        if act == 'f':
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 2 and plur_bets == 1:
                plur_f_when_3b = 'folded_when_3bet'
            bet_order.pop(0)
        elif act.isnumeric():
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 0:
                plur_r_pf = 'pluribus_opens_pf'
                plur_r_size = act
                plur_bets += 1
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 1:
                plur_3b_pf = 'pluribus_3bet_pf'
                plur_3b_size = act
                plur_bets += 1
            if bet_order[0] == 'Pluribus' and street == 1 and raise_num == 0:
                plur_bets_flop = 'pluribus_bet_flop'
            bet_order.append(bet_order[0])
            bet_order.pop(0)
            raise_num += 1
        elif act == 'c':
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 1:
                plur_c_open = 'pluribus_flats_open_pf'
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 2 and plur_bets == 1:
                plur_c_when_3b = 'pluribus_flats_when_3b'
            if bet_order[0] == 'Pluribus' and street == 0 and raise_num == 2 and plur_bets == 0:
                plur_cold_c_3b = 'pluribus_cold_called_3b'
            bet_order.append(bet_order[0])
            bet_order.pop(0)
        elif act == 'next_street':
            street += 1
            raise_num = 0
            
    return(plur_r_pf, plur_3b_pf, plur_bets_flop,
           plur_f_when_3b, plur_c_open, plur_c_when_3b, plur_cold_c_3b,
           plur_r_size, plur_3b_size, plur_able_to_3b)

def hand_matrix_pos(rank_hand):
    if len(rank_hand) == 2:
        row = col = card_ranks.get(rank_hand[0])
    else:
        if rank_hand[2] == 's':
            row = card_ranks.get(rank_hand[0])
            col = card_ranks.get(rank_hand[1])
        else:
            col = card_ranks.get(rank_hand[0])
            row = card_ranks.get(rank_hand[1])
    return row, col

def first_raise_position(original_actions):
    try:
        first_raise_ix = re.search('r', original_actions).start()
    except:
        first_raise_ix = 99
    return first_raise_ix # return a switch for the positions since different from dict

#TODO: WRITE FUNCTION THAT GETS TOTAL HIGHEST NUMBER OF SINGLE SUIT ON THE FLOP

In [3]:
hands_path = '/Users/chrismatthews/Dropbox/Poker/pluribus/'
position_dict = {0:'sb', 1:'bb', 2:'utg', 3:'hj', 4:'co', 5:'d'}
open_position_pf = {0:'utg', 1:'hj', 2:'co', 3:'d', 4:'sb', 5:'bb'}
card_ranks = {'A':1, 'K':2, 'Q':3, 'J':4, 'T':5,
              '9':6, '8':7, '7':8, '6':9, '5':10,
              '4':11, '3':12, '2':13}

hand_list = load_hands(hands_path)

In [4]:
splt = split_hand(hand_list[0])
street_actions = get_actions(splt[2])
tmp = splt[-1].replace('\n','').split('|')
bet_order = tmp[2:] + tmp[0:2]
print(splt)
print(street_actions)
print(bet_order,'\n------')

def get_tables_actions(bet_order, street_actions):
    street_names = {0:'pre_flop',1:'flop',2:'turn',3:'river'}
    street = 0
    current_bet_size = 0
    for act in street_actions:
        print(bet_order[0], act, street_names.get(street))
        if act == 'next_street':
            street += 1
            next
        elif act == 'f':
            bet_order.pop(0)
        elif act.isnumeric():
            bet_order.append(bet_order[0])
            bet_order.pop(0)
            raise_amount = int(act) - current_bet_size
            current_bet_size = int(act)
            print(current_bet_size, raise_amount)
        elif act == 'c':
            bet_order.append(bet_order[0])
            bet_order.pop(0)
get_tables_actions(bet_order, street_actions)

['STATE', '67', 'ffffr300r900c/cr2150c/cc/cc', '9sTs|JcTc|Ad8c|3d8h|Qh2s|9d2h/9h4h7s/Js/3h', '-2150|2150|0|0|0|0', 'MrBlonde|MrWhite|MrPink|MrOrange|Pluribus|MrBlue\n']
['f', 'f', 'f', 'f', '300', '900', 'c', 'next_street', 'c', '2150', 'c', 'next_street', 'c', 'c', 'next_street', 'c', 'c']
['MrPink', 'MrOrange', 'Pluribus', 'MrBlue', 'MrBlonde', 'MrWhite'] 
------
MrPink f pre_flop
MrOrange f pre_flop
Pluribus f pre_flop
MrBlue f pre_flop
MrBlonde 300 pre_flop
300 300
MrWhite 900 pre_flop
900 600
MrBlonde c pre_flop
MrWhite next_street pre_flop
MrWhite c flop
MrBlonde 2150 flop
2150 1250
MrWhite c flop
MrBlonde next_street flop
MrBlonde c turn
MrWhite c turn
MrBlonde next_street turn
MrBlonde c river
MrWhite c river


In [5]:
data_list = list()
for item in hand_list:
    if item.startswith('STATE'):
        data_row = list()
        splt = split_hand(item)
        data_row.append(splt[1]) # hand number
        tmp = splt[-1].replace('\n','').split('|')
        data_row.append(position_dict.get(tmp.index('Pluribus'))) # table position
        bet_order = tmp[2:] + tmp[0:2]
        hole_cards = splt[3].split('/')[0].split('|')
        data_row.append(hole_card_simplifier(hole_cards)[tmp.index('Pluribus')])
        data_row.append(hole_cards[tmp.index('Pluribus')]) # natural hole cards
        if len(splt[3].split('/')) > 1: # flop texture
            flop_ranked = flop_texture(splt[3].split('/')[1])
            flop = splt[3].split('/')[1]
        else:
            flop_ranked = 'no_flop'
            flop = 'no_flop'
        data_row.append(flop_ranked)
        data_row.append(flop)
        data_row.append(splt[4].split('|')[tmp.index('Pluribus')]) # won-lost $
        street_actions = get_actions(splt[2])
        plur_acts = get_pluribus_actions(bet_order, street_actions)
        data_row.extend(plur_acts)
        
#         open raise possible
        if ( 
            (tmp.index('Pluribus') == 2) or
            (splt[2].startswith('f') and tmp.index('Pluribus') == 3) or
            (splt[2].startswith('ff') and tmp.index('Pluribus') == 4) or 
            (splt[2].startswith('fff') and tmp.index('Pluribus') == 5) or
            (splt[2].startswith('ffff') and tmp.index('Pluribus') == 0) or
            (splt[2].startswith('ffffc') and tmp.index('Pluribus') == 1)):
            data_row.append('unopened up to Pluribus')
        else:
            data_row.append('opened before Pluribus acted')
        
        matrix_row, matrix_column = hand_matrix_pos(hole_card_simplifier(hole_cards)[tmp.index('Pluribus')])
        data_row.append(matrix_row)
        data_row.append(matrix_column)
        data_row.append(open_position_pf.get(first_raise_position(splt[2])))
        data_row.append(item)
        data_list.append(data_row)
        
    else:
        continue


In [6]:
col_names = ['hand_num', 'position', 'hand_rank', 'actual_hand', 'flop_rank', 'actual_flop',
             'won_lost', 'pf_open', 'pf_3b', 'bet_flop', 'fold_to_3b', 'pf_flat', 'call_3b', 'cold_call_3b',
             'raise_size', '3b_size', 'able_to_3bet_flag', 'unopened_flag',  
             'hand_matrix_row', 'hand_matrix_column', 'position_to_open_pf',
             'original_log_file']
df = pd.DataFrame(data_list, columns=col_names)

In [7]:
print(df.shape)
df.head(5)

(10000, 22)


,hand_num,position,hand_rank,actual_hand,flop_rank,actual_flop,won_lost,pf_open,pf_3b,bet_flop,...,call_3b,cold_call_3b,raise_size,3b_size,able_to_3bet_flag,unopened_flag,hand_matrix_row,hand_matrix_column,position_to_open_pf,original_log_file
0,67,co,Q2o,Qh2s,974t,9h4h7s,0,,,,...,,,,,Pluribus unable to 3bet,unopened up to Pluribus,13,3,sb,STATE:67:ffffr300r900c/cr2150c/cc/cc:9sTs|JcTc...
1,68,hj,K3o,3hKd,A22r,2dAc2s,0,,,,...,,,,,Pluribus unable to 3bet,unopened up to Pluribus,12,2,sb,STATE:68:ffffr300c/r600c/cc/cr1300f:9h8d|5hAs|...
2,69,utg,95o,9s5d,A42t,Ad2c4d,0,,,,...,,,,,Pluribus unable to 3bet,unopened up to Pluribus,10,6,sb,STATE:69:ffffr300c/r450c/r1450c/cc:TcJc|9cAc|9...
3,70,bb,JTo,TdJc,J97t,9c7cJh,-1075,,,,...,,,,,Pluribus able to 3bet,opened before Pluribus acted,5,4,d,STATE:70:fffr250fc/cc/cc/r1075c:5hKd|TdJc|8cJd...
4,71,sb,75o,5s7d,no_flop,no_flop,-50,,,,...,,,,,Pluribus able to 3bet,opened before Pluribus acted,10,8,d,STATE:71:fffr200ff:5s7d|9h3s|7c3d|2dQh|9c5h|Qd...


In [8]:
df.to_csv('pluribus_data.csv', index=False)